#### Step 1: Download '2019-2022 5 IC' as csv
#### Step 2: Use 'works' and 'works_ids' from OpenAlex and '2019-2022 5 IC' to get data similar to '2019-2022 5 IC' that only contains publications of type 'article'
#### Step 3: Count the unique PubMedIds(PMID) from each IC and unique PMID total(which are different because a single PMID can appear in more than one IC)
#### Step 4: Count the 'is_open_data' True values from each IC and the total

In [1]:
import pandas as pd

We want the publications which are articles from google sheets
We have the pmid on google sheets and openalex-works-ids
We have the type on works
We have the work-id on works and works-ids

In [2]:
sheets_df = pd.read_csv('dsst.csv')

In [4]:
import gzip

with gzip.open('csv-files/works.csv.gz') as f:
    works_df = pd.read_csv(f)
with gzip.open('csv-files/works_ids.csv.gz') as f:
    id_df = pd.read_csv(f)

In [5]:
# Openalex id of articles
article_work_ids = works_df.loc[works_df['type'] == 'article', 'id']

In [6]:
# The pmid from the articles
article_pmid = id_df.loc[id_df['work_id'].isin(article_work_ids), 'pmid'].str.extract(r'(\d+)').astype(int)
article_pmid = article_pmid[0]

In [7]:
# The publications on the google sheets that are articles
opendata_articles = sheets_df[sheets_df['PMID'].isin(article_pmid)]

In [8]:
# Unique pmid after grouping ICs
countUnique = opendata_articles.groupby('IC')['PMID'].nunique()

In [34]:
# Articles which are open data
pmidopendata = opendata_articles[opendata_articles['is_open_data']].groupby('IC')['PMID'].nunique()

In [35]:
# All the articles by IC and all articles that are open data by IC
pivot = pd.concat([countUnique, pmidopendata], axis=1)
pivot.columns = ['unique', 'open_data']
pivot.reset_index(inplace=True)

In [36]:
# The percentage of open articles
pivot['%'] = pivot['open_data']/pivot['unique']

In [41]:
# The total, removing pmids that repeat across ICs
final_row = {
    'IC': 'Grand Total',
    'unique': opendata_articles['PMID'].nunique(),
    'open_data': opendata_articles[opendata_articles['is_open_data']]['PMID'].nunique(),
    '%': pivot['%'].mean()
}

In [42]:
pd.concat([pivot, pd.DataFrame([final_row])], ignore_index=True)

,IC,unique,open_data,%
0,NCI,4658,903,0.193860
1,NHGRI,593,171,0.288364
2,NICHD,1053,165,0.156695
3,NIMH,944,134,0.141949
4,NINDS,720,96,0.133333
5,Grand Total,7761,1419,0.182840
